# Homework 1

## *Learning Python Basic and NumPy*

---

### Problem 1: Welcome (20 points)

(a) Write a funbction below that takes as input a string containing a name in the "Last_Name, First_Name" format and prints a welcome message with the first name first and the last name last and tells the person the length of their first name. The function should be able to handle a wrong input and print a meassge:  *Input format error: please use "Last_Name, First_Name"*


In [ ]:
name_str = input('Last_Name, First_Name: ')
my_str = name_str.split(",")
answer = len(my_str)
if answer == 2:
    first = my_str[1]
    last = my_str[0]
    L = len(first.strip())
    msg = first.title() +' '+ last.title() + ' '+'and your first name is ' + str(L) + ' letters in length' 
else:
    msg = "Input format error: please use \"Last_Name, First_Name\""
print(msg)

(b) Test your function below by calling function with following examples:

- "Hamilton, Alexander" should return Alexander Hamilton your first name is 9 letters in length
- "Kennedy, John" should return John Kennedy your first name is 4 letters in length
- " &nbsp;  Obama, &nbsp;   Barack &nbsp;  " should return Barack Obama your first name is 6 letters in length

Note that the extra spaces in the third name must be removed


In [ ]:
welcome("Hamilton, Alexander")
welcome("Kennedy, John")
welcome("  Obama,   Barack   ")
welcome("Ming Fang")

### Problem 2: Moving Average (40 points)

(a) Write a function below that can compute the moving average of time series using a sliding window over an array. This function should take an array of date strings (say arr_date), an array of numbers (say arr_record), and a sliding window (default value 50). It should:

- Return a list of dictionaries for all windows.
- Each dictionary should include the date, average value, min, max, standard deviation at each window.
- Able to handle missing data in time series by replacing missing data with the most recent available data.


In [ ]:
import numpy as np

def moving_avg(arr_date,arr_record, window = 50):
    length = len(arr_date)
    list_dic = []
    if np.isnan(arr_record[0]):
        finite = np.argwhere(np.isfinite(arr_record))
        arr_record[0] = arr_record[finite[0,0]]
        nan = np.argwhere(np.isnan(arr_record))
        for i in range(len(nan)):
            position = nan[i,0]
            arr_record[position] = arr_record[position-1]
        for i in range(window, len(arr_record) + 1):
            my_data = arr_record[i-window:i]
            print(len(my_data))
            my_mean = my_data.mean()
            my_std = my_data.std()
            my_min = my_data.min()
            my_max = my_data.max()
            my_dict = {"date":arr_date[i],"avg":my_mean,"std":my_std,"min":my_min,"max":my_max}
            list_dict.append(my_dict)
            return list_dic

(b) Download SPY daily data (Dec. 31, 2017 to Dec. 31, 2018) from Yahoo! as your test data in a .csv file. Read [reading .csv file example](https://janakiev.com/blog/csv-in-python/) and write a test programming for calling your function. 

In [ ]:
import csv
import numpy as np

file_path = 'C:/Users/Xue Jin/Documents/Fall 2019/FIN5615/'
file_name = file_path + "SPY.csv"

with open(file_name, 'r') as f_in:
    reader = csv.reader(f_in, delimiter = ',')
    header = next(reader)
    data = list(reader)
f_in.close()
    
N = len(data)
date_arr = np.empty(N, dtype = object)
price_arr = np.empty(N)
i = 0

for my_data in data:
    date_arr[i] = my_data[0]
    price_arr[i] = my_data[5]
    i += 1

# test case
mv_avg = moving_avg(date_arr, price_arr)
print("\n Test case 1 when all price data are available:")
print(mv_avg[-1])
print(mv_avg[100])
print(mv_avg[0])

# test case for the last element has nan value
print("\n Test case 2 when the last element has nan value:")
price_arr[-1] = np.nan
mv_avg = moving_avg(date_arr, price_arr)
print(mv_avg[-1])

# test case for the first element has nan value
print("\n Test case 3 when the first element has nan value:")
price_arr[0] = np.nan
mv_avg = moving_avg(date_arr, price_arr)
print(mv_avg[0])

# test case for 100th element has nan value
print("\n Test case 4 when 100th element has nan value:")
price_arr[100] = np.nan
mv_avg = moving_avg(date_arr, price_arr)
print(mv_avg[100])

### Problem 3: Monte Carlo Simulation (40 points)

Stock prices follow a log-normal process: $dS = S(\mu dt + \sigma dz)$, where $\mu$ is a constant growth rate, $\sigma$ is a constant volatility rate, $dt$ is time interveral (1 month = 1/12), and $dz$ is a normal distributed random variable with variance equal to $dt$.

Start from a given $S_0$ (say, 100), we will have $S_t = S_{t-1}*(1+\mu \Delta t + \sigma \sqrt{\Delta t} Z)$ for $t=1,2,...,N$, where $Z$ is a standard normal distributed variable. 


(a) Write a function below that returns a numPy array with $N+1$ stock prices following the algorithm described above given input: $N$, $\mu$, $\sigma$, $\Delta t$ with default value 1 month ($ = 1/12$), and starting price $S_0$ with default value 100.

In [ ]:
import numpy as np

def prices(n, m, s, seed = None, dt=1/12, p0=100):
    np.random.seed(seed)
    a = m*dt
    b = s*sqrt(dt)
    z = np.random.randn(n)
    r = a+b*z
    prices = np.empty(n+1)
    p0 = prices[0]
    prices[i] = prices[i-1]*(1+r[i-1])
    return prices

(b) Generate a sequence of 60 monthly stock prices by calling function created above with $\mu = 6\%, \sigma = 10\%$, $S_0 = 100$. Calculate 60 monthly returns based on this set of prices. Compute the annualized return and annualized standard deviation. 

In [ ]:
my_prices = prices(60, 0.06, 0.1, 0)
p0 = my_prices[0:60]
p1 = my_prices[1:61]
r = (p1-p0)/p0
print(r)

(c) Similar to (b), use the function crated in (a) to generate 100 sample of return data. Calculate the min, max, mean, and standard deviation at each month end (60 months) below. What's your observation of your average monthly return for each month? How about increase the sample size to 1,000? 10,000?

In [ ]:
n = 60
k = 10000
m = 0.06
s = 0.1

my_prices = np.empty((n+1,k))

for j in range(k):
    my_prices[:,j] = prices(n, m, s)
    
p0 = my_prices[0:n,:]
p1 = my_prices[1:n+1,:]
r2 = (p1-p0)/p0
       
r_min = r2.min(axis=1)
r_max = r2.max(axis=1)
r_mean = r2.mean(axis=1)
r_std = r2.std(axis=1)

print("The mean from the distribution should be: " + str(m/12) + "\n") # theoretic value of monthly volatility
print("Our numerical results are: \n")
print(r_mean)

print("\nThe volatility from the distribution should be: " + str(s/np.sqrt(12)) + "\n") # theoretic value of monthly volatility
print("Our numerical results are: \n")
print(r_std)